In [ ]:
import pickle

In [ ]:
filename = "../CodeSearchNet/big_csn_data/CodeSearchNet_Java_Search_Preliminary.pkl"
infile = open(filename,'rb')
new_dict = pickle.load(infile)
infile.close()

In [ ]:
len(new_dict)

In [ ]:
i = -1 #+(100000)*15
print(new_dict[i]['function'])

In [ ]:
len(new_dict[i]['function_tokens'])

## Analyzing Compression

In [ ]:
def getJsonDataLineByLine(JsonFile):
    data = []
    i = 0
    with open(JsonFile) as f:
        for line in f:
            i += 1
            line = line.strip()
            if line[0] == '[':
                line = line[1:]
            elif line[-1] == ']':
                line = line[:-1]
            if line[-1] == ',':
                line = line[:-1]
            try:
                data.append(json.loads(line))
                if i%10000 == 0: print(i, "success")
            except:
                data.append('''{"classBodyDeclaration": ""}''')
                print(i, "failure")
                print(line)
    return data

# Visualize Tree

In [ ]:
def travarse(tree):
    if 'type' in tree:
        print(tree['text'], end=" ")
        if tree['text'] in [';', '{', '}']:
            print()
        return
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        for child in children:
            travarse(child)

In [ ]:
for i in range(100):
    print(tree2code(tree_data[i],0)[0])

In [ ]:
def printCode(tree):
    code, _ = tree2code(tree, 0)
    print(code)
    print("Count:", len(code.split()))

In [ ]:
ls ../csn_big_only_trees/

In [ ]:
uncompressed_data = getJsonData('../csn_big_only_trees/csn_big_1.json')

In [ ]:
compressed_data = getJsonData('../CodeSearchNet/train_test_valid_data_10k/java_train_1u_10k_compressed.json')

In [ ]:
bef_comp = 0
aft_comp = 0
for i in range(len(compressed_data)):
    code, _ = tree2code(uncompressed_data[i], 0)
    num_tokens = len(code.split())
    compressed_tokens = len(compressed_data[i]['code'].split())
    bef_comp += num_tokens
    aft_comp += compressed_tokens
    if not compressed_tokens:
        aft_comp += num_tokens

print(1-aft_comp/bef_comp)

In [ ]:
i = 205

In [ ]:
code, _ = tree2code(uncompressed_data[i], 0)
num_tokens = len(code.split())
print(code)
print(num_tokens)
print("================================")
print(compressed_data[i]['code'])
compressed_tokens = len(compressed_data[i]['code'].split())
print(compressed_tokens)
print("Compression Ratio:", compressed_tokens/num_tokens)
i+=1

In [ ]:
0.2104572106016639

## How many of the compressed tokens were present in the top 10k train data?

In [ ]:
abstract_code_dict = getJsonData('../CodeSearchNet/abstract_code_dict_500k_w_child.json')

In [ ]:
abstract_code_keys = sorted(abstract_code_dict, key = lambda x:abstract_code_dict[x]['count'], reverse=True)

In [ ]:
abstract_code_dict[abstract_code_keys[110]]

In [ ]:
most_freq_25k = [abstract_code_dict[abstract_code_keys[i]]['compressed_name'] for i in range(25000)]

In [ ]:
most_freq_10k = [abstract_code_dict[abstract_code_keys[i]]['compressed_name'] for i in range(10000)]

In [ ]:
most_freq_25k[:1000]

In [ ]:
#!mkdir ../CodeSearchNet/25k
#!mv ../CodeSearchNet/train-test-valid_25kf.zip ../CodeSearchNet/25k/
#!unzip ../CodeSearchNet/25k/train-test-valid_25kf.zip
!ls ../CodeSearchNet/train_test_valid_data_10k/

In [ ]:
import glob
dirs = glob.glob('../CodeSearchNet/train_test_valid_data_10k_500k/*_train_*')
dirs

In [ ]:
freq_dict = {}
for path in dirs:
    dataset = getJsonData(path)
    for data in dataset:
        code = data['code']
        code_tokens = code.split()
        for token in code_tokens:
            if token not in freq_dict:
                freq_dict[token] = 0
            freq_dict[token] += 1

In [ ]:
len(freq_dict)

In [ ]:
# 25k compressed total tokens: 1024352
# 10k compressed total tokens: 1160247

In [ ]:
data_freq_list = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)

In [ ]:
top_data_freq_list = data_freq_list[:30000]

In [ ]:
top_data_freq_list[:1000]

In [ ]:
common_tokens = list(set(top_data_freq_list) & set(most_freq_10k))

In [ ]:
len(common_tokens)

In [ ]:
len(top_data_freq_list)

Dumping common union vocabs

In [ ]:
list_10k_10k = []

In [ ]:
for token in top_data_freq_list:
    list_10k_10k.append({'token':token, 'count':freq_dict[token]})

In [ ]:
additional_compressed_tokens = list(set(most_freq_10k) - set(top_data_freq_list))
len(additional_compressed_tokens)

In [ ]:
for token in additional_compressed_tokens:
    if token in freq_dict:
        list_10k_10k.append({'token':token, 'count':freq_dict[token]})

In [ ]:
len(list_10k_10k)

In [ ]:
dump_dict(list_10k_10k, "list_10k_10k.json")

Testing out data outside vocab

In [ ]:
not_in_vocab = list(set(most_freq_10k) - set(top_data_freq_list))
len(not_in_vocab)

In [ ]:
not_in_vocab[:100]

In [ ]:
for f in not_in_vocab:
    print(f,"||", func2abs[f],"||", abstract_code_dict[func2abs[f]]['count'])

In [ ]:
for f in common_tokens[:1000]:
    print(f,"||", func2abs[f],"||", abstract_code_dict[func2abs[f]]['count'])

In [ ]:
abstract_code_dict[abstract_code_keys[10000]]

In [ ]:
abstract_code_dict['( ( AbstractISUPParameter ) $id0 ) . decode ( $id1 ) ;']

In [ ]:
'$F2570015' in common_tokens

In [ ]:
len(abstract_code_keys)
#1144264

In [ ]:
i = -1
abstract_code_keys2 = []
for key in abstract_code_keys:
    i+=1
    parent = key + " ;" 
    if parent in abstract_code_keys:
        if not i%1000: print(i)
        if abstract_code_dict[key]['count'] != abstract_code_dict[parent]['count']:
            abstract_code_keys2.append(key)

abstract_code_keys = abstract_code_keys2
len(abstract_code_keys)

In [ ]:
i = -1
abstract_code_keys2 = []
for key in abstract_code_keys:
    i+=1
    if not i%100: print(i)
    #abstract_code_keys.remove(key)
    abstract_code_keys2.append(key)

## Removing child counts

In [ ]:
min_freq = freq_dict[data_freq_list[30000]]
min_freq

In [ ]:
for discard_after in range(len(abstract_code_keys)):
    key = abstract_code_keys[discard_after]
    if abstract_code_dict[key]['count'] <= min_freq:
        break
discard_after

In [ ]:
func2abs = {}

for key in abstract_code_dict:
    func2abs[abstract_code_dict[key]['compressed_name']] = key

In [ ]:
accepted_abstract_keys = abstract_code_keys[:discard_after]
abstract_code_dict_modified = copy.deepcopy(abstract_code_dict)

In [ ]:
for key in accepted_abstract_keys:
    children = abstract_code_dict[key]['children']
    if children:
        count = abstract_code_dict[key]['count']
        for child in children:
            try:
                child_key = func2abs[child]
            except:
                print("parent:", abstract_code_dict_modified[key], "child:", child)
            abstract_code_dict_modified[child_key]['count'] -= count

In [ ]:
abstract_code_keys = sorted(abstract_code_dict_modified, key = lambda x:abstract_code_dict_modified[x]['count'], reverse=True)

In [ ]:
for discard_after in range(len(abstract_code_keys)):
    key = abstract_code_keys[discard_after]
    if abstract_code_dict_modified[key]['count'] <= min_freq:
        break
discard_after

In [ ]:
accepted_abstract_keys = abstract_code_keys[:discard_after]

In [ ]:
len(top_data_freq_list)

In [ ]:
abstract_code_dict2 = getJsonData('../CodeSearchNet/abstract_code_dict_500k.json')

In [ ]:
accepted_abstract_funcs = [ abstract_code_dict2[x]['compressed_name'] for x in accepted_abstract_keys ]

In [ ]:
not_in_vocab = list(set(accepted_abstract_funcs) & set(top_data_freq_list))
len(not_in_vocab)

## Testing new model

In [8]:
import json
import copy
def getJsonData(JsonFile):
    with open(JsonFile, encoding="utf8") as f:
        data = json.load(f)
    return data
    
def dump_dict(data, name):
    with open(name, 'w', encoding="utf8") as f:
        json.dump(data, f)

In [3]:
import ast
def tree2code(tree, indentation):
    code = ""
    parent = list(tree.keys())[0]
    children = tree[parent]
    if 'type' in tree:
        code = tree['text'] + ' '
        if tree['text'] in [';', '{', '}']:
            code += '\n'
            if tree['text'] == '{':
                indentation += 1
            elif tree['text'] == '}':
                indentation -= 1
            code += '\t'*indentation
        #return code, indentation
    elif len(children)==1 and parent == "literal":
        child = children[0]
        child_text = child["text"]
        #type(ast.literal_eval(child_text))
        try:
            if child_text in ['null', 'true', 'false']:
                code = child_text + " "
            elif type(ast.literal_eval(child_text)) == type("STRING"):
                code = "STRING00 "
            elif type(ast.literal_eval(child_text)) == type(1):
                code = "INT00 "
            elif type(ast.literal_eval(child_text)) == type(1.1):
                code = "FLOAT00 "
        except:
            code = "NUM00 "
    else:
        root = list(tree.keys())[0]
        children = tree[root]
        for child in children:
            t, indentation = tree2code(child, indentation)
            code += t
    return code, indentation

In [4]:
import ast
def get_compressed_func():
    global compression_number
    compression_number += 1
    return "$F"+str(compression_number)

In [5]:
def build_dict(tree):
    var_list = []
    global abstract_code_dict_copy
    global abstract_code_keys
    
    
    if 'text' in tree:
        code = tree["text"] + " "
        #print("**:\t",code, 1)
        return (code, 1, 0, [], None)
    else:
        parent = list(tree.keys())[0]
        children = tree[parent]
        if len(children) == 0:
            return ("", 0, 0, [], None)
        
        code = ""
        compressed_func = None
        leaf_count = 0
        id_count = 0
        children_list = []
        #leaf_list = []
        
        '''
        and not (len(children) == 2 and 'text' in children[1] and children[1]['text'] == ';')\
        and not (len(children) == 3 and 'text' in children[0] and 'text' in children[2] and \
                 ((children[0]['text'] == '(' and children[2]['text'] == ')') or 
                  (children[0]['text'] == '{' and children[2]['text'] == '}') or
                  (children[0]['text'] == '[' and children[2]['text'] == ']')))
        '''
        
        if len(children) > 1:
            for child in children:
                (c, n, i, vl, f) = build_dict(child)
                code += c
                leaf_count += n
                id_count += i
                var_list += [v for v in vl if v not in var_list]
                if f: children_list.append(f)
                #else: leaf_list += code.split()
                
            if (var_list and leaf_count - len(var_list) >= 4) or (not var_list and leaf_count > 1) :
                abstract_code = code.split()
                for i in range(len(abstract_code)):
                    if abstract_code[i] in var_list:
                        abstract_code[i] = "$id{}".format(var_list.index(abstract_code[i]))
                abstract_code = ' '.join(abstract_code)
                
                if abstract_code not in abstract_code_dict:
                    compressed_func = get_compressed_func()
                    abstract_code_dict[abstract_code] = {
                        'count': 0, 
                        'compressed_name': compressed_func,
                        'num_params': len(var_list),
                        'children': children_list,
                        #'leaves': leaf_list
                    }
                else:
                    compressed_func = abstract_code_dict[abstract_code]['compressed_name']
                
                abstract_code_dict[abstract_code]['count'] += 1
#                 children_list.insert(0, compressed_func)
                
            return (code, leaf_count, id_count, var_list, compressed_func)
        
        if parent == "literal":
            child = children[0]
            child_text = child["text"]
            try:
                if child_text in ['null', 'true', 'false']:
                    pass
                elif type(ast.literal_eval(child_text)) == type("STRING"):
                    children[0]["text"] = "STRING00"
                elif type(ast.literal_eval(child_text)) == type(1):
                    children[0]["text"] = "INT00"
                elif type(ast.literal_eval(child_text)) == type(1.1):
                    children[0]["text"] = "FLOAT00"
            except:
                children[0]["text"] = "NUM00"
            code = children[0]["text"] + " "
#==================================================================
        elif parent == "primary" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
            code = text + " "
        elif parent == "variableDeclaratorId" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
            code = text + " "
#=================================================================
        else:
            (code, leaf_count, i, vl, compressed_func) = build_dict(children[0])
            id_count += i
            var_list += [v for v in vl if v not in var_list]
        
        return (code, leaf_count, id_count, var_list, compressed_func)

In [6]:
def compress(tree):
    var_list = []
    global abstract_code_dict_copy
    global abstract_code_keys
    
    if 'text' in tree:
        code = tree["text"] + " "
        #print("**:\t",code, 1)
        return (code, 1, 0, [])
    else:
        parent = list(tree.keys())[0]
        children = tree[parent]
        if len(children) == 0:
            return ("", 0, 0, [])
        
        code = ""
        compressed_func = None
        leaf_count = 0
        id_count = 0
        children_list = []
        
        if len(children) > 1:
            for child in children:
                (c, n, i, vl) = compress(child)
                code += c
                leaf_count += n
                id_count += i
                var_list += [v for v in vl if v not in var_list]
                #if f: children_list.append(f)
                #else: leaf_list += code.split()
                
            if (var_list and leaf_count - len(var_list) >= 4) or (not var_list and leaf_count > 1) :
                abstract_code = code.split()
                for i in range(len(abstract_code)):
                    if abstract_code[i] in var_list:
                        abstract_code[i] = "$id{}".format(var_list.index(abstract_code[i]))
                abstract_code = ' '.join(abstract_code)
                
                #---------------------------------------------------
                if abstract_code in abstract_code_keys:
                    #print("compressing!!")
                    compressed_func = abstract_code_dict_copy[abstract_code]['compressed_name']
                    compressed_func += ' ( ' +' , '.join(var_list)+' )' if var_list else ''
                    tree[parent] = [{"type": "compressed", "text": compressed_func}]
                #---------------------------------------------------

            return (code, leaf_count, id_count, var_list)
        
        if parent == "literal":
            child = children[0]
            child_text = child["text"]
            try:
                if child_text in ['null', 'true', 'false']:
                    pass
                elif type(ast.literal_eval(child_text)) == type("STRING"):
                    children[0]["text"] = "STRING00"
                elif type(ast.literal_eval(child_text)) == type(1):
                    children[0]["text"] = "INT00"
                elif type(ast.literal_eval(child_text)) == type(1.1):
                    children[0]["text"] = "FLOAT00"
            except:
                children[0]["text"] = "NUM00"
            code = children[0]["text"] + " "
            
        elif parent == "primary" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
            code = text + " "
        elif parent == "variableDeclaratorId" and 'type' in children[0]:
            id_count += 1
            text = children[0]["text"]
            if text not in var_list:
                var_list.append(text)
            code = text + " "
        else:
            (code, leaf_count, i, vl) = compress(children[0])
            id_count += i
            var_list += [v for v in vl if v not in var_list]
        
        return (code, leaf_count, id_count, var_list)

In [9]:
## Loading one training data
tree_data = getJsonData("../test_data/java_train_3.json")

### Building Dictionary
[Parent Child](#Parent-child)

In [ ]:
# Num items first: 411762
# After removing children low frequency than 10,000th in freq_dict: 4810
# After deducing parent frequency from children: 2922
# After 

In [6]:
abstract_code_dict = {}
compression_number = 0

In [7]:
i = 0
for data in tree_data:
    if not i%5000: print(i)
    build_dict(data)
    i += 1
print(i)

0
5000
10000
15000
20000
25000
30000


In [8]:
dump_dict(abstract_code_dict, '../test_data/abstract_code_dict.json')

In [14]:
## build freq_dict
freq_dict_original = {}
uncompressed_codes = []

for i in range(len(tree_data)):
    if i%5000 == 0: print("[", i, "]")
    code, _ = tree2code(tree_data[i], 0)
    uncompressed_codes.append({"code": code})
    tokens = code.split()

    for token in tokens:
        if token not in freq_dict_original: freq_dict_original[token] = 0
        freq_dict_original[token] += 1
print("[", i, "]")

[ 0 ]
[ 5000 ]
[ 10000 ]
[ 15000 ]
[ 20000 ]
[ 25000 ]
[ 29999 ]


In [15]:
dump_dict(freq_dict_original, '../test_data/freq_dict.json')

In [17]:
dump_dict(uncompressed_codes, "../test_data/uncompressed_codes_no_literal_train_3.json")

In [11]:
abstract_code_dict = getJsonData('../test_data/abstract_code_dict.json')

In [12]:
func2abs = {}

for key in abstract_code_dict:
    func2abs[abstract_code_dict[key]['compressed_name']] = key

In [13]:
freq_dict_original = getJsonData('../test_data/freq_dict.json')

### For Reruns, run from here

In [14]:
vocab_size = 30000

In [15]:
import copy
abstract_code_dict_copy = copy.deepcopy(abstract_code_dict)
abstract_code_keys = sorted(abstract_code_dict_copy, key = lambda x:abstract_code_dict_copy[x]['count'], reverse=True)
len(abstract_code_keys)

411762

In [16]:
freq_dict = copy.deepcopy(freq_dict_original)
most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
len(most_freq)

82769

In [17]:
min_freq = freq_dict[most_freq[vocab_size]]
most_freq[vocab_size], min_freq
# 10k : ('nColors', 11)
# 30k: ('ceMask_', 4)

('ceMask_', 4)

In [18]:
# Tokens that are less than the min-count, will not appear on the final vocab

for key in list(abstract_code_dict_copy):
    if abstract_code_dict_copy[key]['count'] < min_freq:
        del abstract_code_dict_copy[key]
abstract_code_keys = sorted(abstract_code_dict_copy, key = lambda x:abstract_code_dict_copy[x]['count'], reverse=True)
len(abstract_code_keys)

21195

#### Removing the frequency of every parent from their children, because if parent exist in the vocabulary, child will not

for key in abstract_code_keys:
    children = abstract_code_dict_copy[key]['children']
    count = abstract_code_dict_copy[key]['count']
    for child in children:
        child_key = func2abs[child]
        if child_key in abstract_code_dict_copy:
            abstract_code_dict_copy[child_key]['count'] -= count

##### Removing again all tokens with less frequent than min-count

for key in list(abstract_code_dict_copy):
    if abstract_code_dict_copy[key]['count'] < min_freq:
        del abstract_code_dict_copy[key]
len(abstract_code_dict_copy)

abstract_code_keys = sorted(abstract_code_dict_copy, key = lambda x:abstract_code_dict_copy[x]['count'], reverse=True)
len(abstract_code_keys)

In [ ]:
### 10k
# Before operation total keys: 4810
# After decreasing child count and removing lower than min_count: 2922
# Functions survived after addidngg in freq_dict: 2001
# Ended up after compression: 1989
# Compressio Ratio: 0.8567161502728318

### 30k
# Before operation total keys: 
# After decreasing child count and removing lower than min_count: 2776
# Functions survived after addidngg in freq_dict: 2770
# Ended up after compression:
# Compression Ratio: 0.8987381335587129

### Compress child and token together: 10k
# Coverage: 0.9927927927927928
# Functions survived after addidngg in freq_dict: 2220
# Ended up after compression: 2204
# Compression Ration: 0.8523115931079496

### Compress child and token together: 30k
# Coverage: 
# Functions survived after addidngg in freq_dict: 6663
# Ended up after compression: 
# Compression Ration: 

In [19]:
len(freq_dict)

82769

for key in abstract_code_keys:
    func = abstract_code_dict_copy[key]['compressed_name']
    freq_dict[func] = abstract_code_dict_copy[key]['count']
len(freq_dict)

In [45]:
freq_dict = copy.deepcopy(freq_dict_original)

#### Instead of reducing the compressed child frequency before, we do it with removing other tokens
#### Recursion

i = 0
accepted_functions = []
for key in abstract_code_keys:
    if not i%500: print(i, ":", len(accepted_functions))
    i+=1
    func = abstract_code_dict_copy[key]['compressed_name']
    freq_dict[func] = abstract_code_dict_copy[key]['count']
    most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
    accepted_functions = list(set(func2abs) & set(most_freq[:vocab_size]))
    
    if func in most_freq[:vocab_size]:
        count = abstract_code_dict_copy[key]['count']
        children = abstract_code_dict_copy[key]['children']
        accepted_functions.append(func)
        tokens = key.split()
        
        # Should be recursive
        nodes_to_visit = [func]
        while nodes_to_visit:
            currentnode = nodes_to_visit.pop(0)
            children = abstract_code_dict_copy[func2abs[currentnode]]['children']
            nodes_to_visit = children + nodes_to_visit
            if children:
                for child in children:
                    if child in freq_dict:
                        freq_dict[child] -= count
                    else:
                        print(child, "is less frequent than parent", func)
            else:
                for token in tokens:
                    if not token.startswith("$id") and token in freq_dict:
                        freq_dict[token] -= count
    

most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
accepted_functions = list(set(func2abs) & set(most_freq[:vocab_size]))
print(i, ":", len(accepted_functions))

In [20]:
## Instead of reducing the compressed child frequency before, we do it with removing other tokens

i = 0
accepted_functions = []
for key in abstract_code_keys:
    if not i%500: print(i, ":", len(accepted_functions))
    i+=1
    func = abstract_code_dict_copy[key]['compressed_name']
    freq_dict[func] = abstract_code_dict_copy[key]['count']
    most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
    accepted_functions = list(set(func2abs) & set(most_freq[:vocab_size]))
    
    if func in most_freq[:vocab_size]:
        count = abstract_code_dict_copy[key]['count']
        children = abstract_code_dict_copy[key]['children']
        accepted_functions.append(func)
        tokens = key.split()
        
        # Should be recursive
        if children:
            for child in children:
                if child in freq_dict:
                    freq_dict[child] -= count
                else:
                    print(child, "is less frequent than parent", func)
        else:
            for token in tokens:
                if not token.startswith("$id") and token in freq_dict:
                    freq_dict[token] -= count

most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
accepted_functions = list(set(func2abs) & set(most_freq[:vocab_size]))
print(i, ":", len(accepted_functions))

0 : 0
$F356864 is less frequent than parent $F356865
$F355549 is less frequent than parent $F355550
$F355283 is less frequent than parent $F355287
$F11431 is less frequent than parent $F11432
500 : 370
$F113753 is less frequent than parent $F113754
1000 : 740
1500 : 1103
2000 : 1432
2500 : 1773
$F113618 is less frequent than parent $F113619
3000 : 2115
3500 : 2458
$F28782 is less frequent than parent $F28783
4000 : 2766
4500 : 3099
$F113672 is less frequent than parent $F113673
5000 : 3412
5500 : 3714
6000 : 4021
6500 : 4366
7000 : 4666
7500 : 4860
8000 : 5182
8500 : 5454
9000 : 5705
9500 : 5970
$F192495 is less frequent than parent $F192496
10000 : 6221
$F289515 is less frequent than parent $F289516
10500 : 6453
11000 : 6635
11500 : 6663
12000 : 6663
12500 : 6663
13000 : 6663
13500 : 6663
14000 : 6663
14500 : 6663
15000 : 6663


KeyboardInterrupt: 

#### Adding one compressed token at a time to the dataset, reducing the frequency for all of it's tokens
#### It's not entirely accurate because we cannot consider the identifier numbers
#### But it's okay because we are taking an upper bound, so less compress tokens are fine, more are not

i = 0
accepted_functions = []
for key in abstract_code_keys:
    if not i%1000: print(i)
    i+=1
    most_freq = sorted(freq_dict, key = lambda x:freq_dict[x], reverse=True)
    count = abstract_code_dict_copy[key]['count']
    func = abstract_code_dict_copy[key]['compressed_name']
    if func in most_freq[:vocab_size]:
        accepted_functions.append(func)
        tokens = key.split()
        for token in tokens:
            if not token.startswith("$id") and token in freq_dict:
                freq_dict[token] -= count
print(i)
len(accepted_functions)

In [21]:
# accepted_fucnctions = list(set([abstract_code_dict_copy[x]['compressed_name'] for x in abstract_code_dict_copy]))
# ## Suggested functions in our estimation

# accepted_functions = done_funcs
len(accepted_functions)

6663

In [22]:
accepted_functions[:10]

['$F374504',
 '$F410006',
 '$F168134',
 '$F409064',
 '$F50242',
 '$F184471',
 '$F77718',
 '$F4569',
 '$F383217',
 '$F105243']

In [23]:
freq_dict[most_freq[vocab_size]]

5

In [24]:
to_be_compressed = []
for token in accepted_functions:
    to_be_compressed.append(func2abs[token])

In [25]:
len(to_be_compressed), to_be_compressed[0]

(6663, '< Page < LiveOutputInner > >')

In [26]:
del abstract_code_keys
abstract_code_keys = to_be_compressed

In [27]:
len(abstract_code_keys)

6663

#### Testing Single compression
i = 14
data = copy.deepcopy(tree_data[i])
uncompressed_code = tree2code(data, 0)[0]
before_tokens = len(uncompressed_code.split())
print(uncompressed_code, "\n", before_tokens)

(code, leaf_count, id_count, var_list) = compress(data)
code = tree2code(data, 0)[0]
after_tokens = len(code.split())
print(code, "\n", after_tokens)
print("Compression Ratio: ", after_tokens/before_tokens)

In [28]:
dump_dict(to_be_compressed, "to_be_compressed_30k.json")

In [29]:
abstract_code_keys = getJsonData("to_be_compressed_30k.json")

In [30]:
compressed_freq_dict = {}
compressed_codes = []
total_tokens_before = 0
total_tokens_after = 0
for i in range(len(tree_data)):
    try:
        tree_copy = copy.deepcopy(tree_data[i])
        code, _ = tree2code(tree_copy, 0)
        total_tokens_before += len(code.split())
        (code, leaf_count, id_count, var_list) = compress(tree_copy)
        code, _ = tree2code(tree_copy, 0)
    except Exception as e:
        print(e)
        code, _ = tree2code(tree_data[i], 0)
    compressed_code = {"code": code}
    tokens = code.split()
    total_tokens_after += len(tokens)
    for token in tokens:
        if token not in compressed_freq_dict:
            compressed_freq_dict[token] = 0
        compressed_freq_dict[token] += 1
    compressed_codes.append(compressed_code)
    if i%5000 == 0: print("[", i, "] :", total_tokens_after/total_tokens_before) 
print("[", i, "] :", total_tokens_after/total_tokens_before)

[ 0 ] : 0.8095238095238095
[ 5000 ] : 0.8564257524562994
[ 10000 ] : 0.856249825302981
[ 15000 ] : 0.8523927947581553
[ 20000 ] : 0.8569750104397331
[ 25000 ] : 0.8424060075147535
[ 29999 ] : 0.8120006470466895


In [31]:
dump_dict(compressed_codes, '../test_data/compressed_codes_train_3_30k.json')

In [32]:
compressed_most_freq = sorted(compressed_freq_dict, key = lambda x:compressed_freq_dict[x], reverse=True)
len(compressed_most_freq)

89140

In [34]:
common_tokens_after_compression = list(set(compressed_most_freq[:vocab_size]) & set(accepted_functions))
len(common_tokens_after_compression)

6607

In [35]:
len(common_tokens_after_compression)/len(accepted_functions)

0.9915953774576017

In [38]:
len(accepted_functions)

2217

In [44]:
compressed_most_freq[:30]

[')',
 '(',
 ',',
 ';',
 '.',
 '{',
 '}',
 'NUM00',
 '=',
 'if',
 'public',
 'new',
 'String',
 'return',
 '<',
 '>',
 'null',
 'int',
 '==',
 '+',
 'i',
 '[',
 ']',
 'this',
 'else',
 '!=',
 ':',
 '$F1',
 'final',
 '-']

In [32]:
oov = list(set(accepted_functions) - set(compressed_most_freq[:vocab_size]))
len(oov)

16

In [33]:
for o in oov:
    a = abstract_code_dict[func2abs[o]]
    print(func2abs[o], a)

{ $id0 . ifModifiedSince = null ; } {'count': 22, 'compressed_name': '$F355291', 'num_params': 1, 'children': []}
ServiceResponse < Void > {'count': 584, 'compressed_name': '$F355581', 'num_params': 0, 'children': ['$F19311']}
long $id0 = $id1 . currentTimeMillis ( ) {'count': 67, 'compressed_name': '$F11431', 'num_params': 2, 'children': []}
{ $id0 . lastModified = null ; } {'count': 47, 'compressed_name': '$F356833', 'num_params': 1, 'children': []}
$id0 != null ? $id0 . skip ( ) : null {'count': 17, 'compressed_name': '$F361252', 'num_params': 1, 'children': []}
ServiceResponse < OperationStatus > {'count': 190, 'compressed_name': '$F361296', 'num_params': 0, 'children': ['$F361287']}
boolean $id0 = $id1 . $id0 ( ) {'count': 20, 'compressed_name': '$F92224', 'num_params': 2, 'children': []}
{ $id0 . ifUnmodifiedSince = null ; } {'count': 22, 'compressed_name': '$F355294', 'num_params': 1, 'children': []}
{ $id0 . ocpDate = null ; } {'count': 59, 'compressed_name': '$F355283', 'num_p

In [ ]:
'$F406149' in compressed_freq_dict

In [ ]:
common_tokens_after_compression[0]

In [ ]:
# Vocab 10k, compression ratio: 0.8567161502728318
# vocab 30k, compression ratio: 0.8987381335587129